<a href="https://colab.research.google.com/github/Duncanswilson/gpt4-code-generation/blob/main/cifar_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install required libraries
!pip install -q tensorflow numpy

# Step 2: Connect to a TPU runtime
%tensorflow_version 2.x
import tensorflow as tf
import os
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
except KeyError:
 print('TPU not found')

# Step 3: Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Step 4: Preprocess the data
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

# Step 5: Build the DNN model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

with strategy.scope():
    model = create_model()

# Step 6: Train the model on the TPU
model.fit(x_train, y_train, epochs=25, batch_size=256, validation_data=(x_test, y_test))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found TPU at: grpc://10.3.127.186:8470


Epoch 1/25
196/196 [==============================] - 12s 40ms/step - loss: 1.7867 - accuracy: 0.3448 - val_loss: 1.5501 - val_accuracy: 0.4324
Epoch 2/25
196/196 [==============================] - 4s 23ms/step - loss: 1.4515 - accuracy: 0.4767 - val_loss: 1.3774 - val_accuracy: 0.5058
Epoch 3/25
196/196 [==============================] - 5s 24ms/step - loss: 1.3258 - accuracy: 0.5264 - val_loss: 1.2619 - val_accuracy: 0.5466
Epoch 4/25
196/196 [==============================] - 5s 24ms/step - loss: 1.2415 - accuracy: 0.5607 - val_loss: 1.2428 - val_accuracy: 0.5555
Epoch 5/25
196/196 [==============================] - 4s 22ms/step - loss: 1.1622 - accuracy: 0.5891 - val_loss: 1.1631 - val_accuracy: 0.5837
Epoch 6/25
196/196 [==============================] - 4s 22ms/step - loss: 1.0966 - accuracy: 0.6143 - val_loss: 1.0756 - val_accuracy: 0.6157
Epoch 7/25
196/196 [==============================] - 5s 23ms/step - loss: 1.0282 - accuracy: 0.6378 - val_loss: 1.0399 - val_accuracy: 0.632

In [2]:
# Step 1: Install the required libraries
!pip install -q tf2onnx

# Step 2: Save the TensorFlow model to a SavedModel format
model.save('cifar_model')

# Step 3: Convert the TensorFlow model to ONNX format
import tf2onnx

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=None, opset=13)
onnx_model_path = 'cifar_model.onnx'

# Step 4: Save the ONNX model to a file
with open(onnx_model_path, 'wb') as f:
    f.write(model_proto.SerializeToString())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


ValueError: ignored